In [149]:
import numpy as np
import os
from pathlib import Path
paths_list=[]
labels_list=[]
for i in range (1, 10):
    for j in range(0, 288):
        if os.path.isfile(f"Data sets 2a 0.0-4.0s 250Hz/data/A{str(i).rjust(2, '0')}TE{str(j).rjust(4, '0')}.npy"):
            paths_list.append(f"Data sets 2a 0.0-4.0s 250Hz/data/A{str(i).rjust(2, '0')}TE{str(j).rjust(4, '0')}.npy")
for i in range (1, 10):
    for j in range(0, 288):
        if os.path.isfile(f"Data sets 2a 0.0-4.0s 250Hz/data/A{str(i).rjust(2, '0')}EE{str(j).rjust(4, '0')}.npy"):
            paths_list.append(f"Data sets 2a 0.0-4.0s 250Hz/data/A{str(i).rjust(2, '0')}EE{str(j).rjust(4, '0')}.npy")
labels_list = np.load('Data sets 2a 0.0-4.0s 250Hz/labels_list.npy')
print(len(paths_list))
print(len(labels_list))

5184
5184


In [129]:
from sklearn import model_selection
x_train, x_test_raw, y_train, y_test_raw = model_selection.train_test_split(
    paths_list, labels_list, test_size=0.2, random_state=42, shuffle=True
)
x_evaluate, x_test, y_evaluate, y_test = model_selection.train_test_split(
    x_test_raw, y_test_raw, test_size=0.5, random_state=42, shuffle=True
)

In [137]:
import tensorflow as tf
import math
class Datasets2aSequence(tf.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        low = idx * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + self.batch_size, len(self.x))
        batch_x = self.x[low:high]
        batch_y = self.y[low:high]

        return np.array([np.moveaxis(np.load(file_name),0,1) for file_name in batch_x]), np.array(batch_y)

In [117]:
def InitialInceptionModule(input):
    maxpooling1d = MaxPooling1D(pool_size=1)(input) 
    maxpooling1dconv1d = Conv1D(filters=48, kernel_size=1, strides=1)(maxpooling1d)
    bottleneck = Conv1D(48, 1, strides=1)(input)    
    conv1d1 = Conv1D(48, 25, strides=1, padding='causal')(bottleneck)
    conv1d2 = Conv1D(48, 75, strides=1, padding='causal')(bottleneck)
    conv1d3 = Conv1D(48, 125, strides=1, padding='causal')(bottleneck)
    conv1d4 = Conv1D(48, 175, strides=1, padding='causal')(bottleneck)
    conv1d5 = Conv1D(48, 225, strides=1, padding='causal')(bottleneck)
    concatenateResult = concatenate([conv1d1,conv1d2,conv1d3,conv1d4,conv1d5,maxpooling1dconv1d],axis=2)
    batchNormalization = BatchNormalization()(concatenateResult)
    activation = Activation('relu')(batchNormalization)
    return activation

In [118]:
def IntermeidateInceptionModule(input):
    maxpooling1d = MaxPooling1D(pool_size=1)(input) 
    maxpooling1dconv1d = Conv1D(filters=48, kernel_size=1, strides=1)(maxpooling1d)
    bottleneck = Conv1D(48, 1, strides=1)(input)    
    conv1d1 = Conv1D(48, 25, strides=1, padding='causal')(bottleneck)
    conv1d2 = Conv1D(48, 75, strides=1, padding='causal')(bottleneck)
    conv1d3 = Conv1D(48, 125, strides=1, padding='causal')(bottleneck)
    conv1d4 = Conv1D(48, 175, strides=1, padding='causal')(bottleneck)
    conv1d5 = Conv1D(48, 225, strides=1, padding='causal')(bottleneck)
    concatenateResult = concatenate([conv1d1,conv1d2,conv1d3,conv1d4,conv1d5,maxpooling1dconv1d],axis=2)
    batchNormalization = BatchNormalization()(concatenateResult)
    activation = Activation('relu')(batchNormalization)
    return activation

In [121]:
def ResidualModel(input):
    conv1d = Conv1D(288, 1, strides=1)(input)
    batchNormalization = BatchNormalization()(conv1d)
    activation = Activation('relu')(batchNormalization)
    return activation

In [145]:
from tensorflow.keras.layers import Conv1D,MaxPooling1D,AveragePooling1D,Activation,BatchNormalization,Dense,Dropout,Flatten,Input,Add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.backend import clear_session
from tensorflow.keras import optimizers
from keras.callbacks import ReduceLROnPlateau
num_classes = 4
sgd = optimizers.SGD(learning_rate=0.005)
def EEGInception():
    clear_session()
    kerasInput= Input((1000,22))
    residualModel1 = ResidualModel(kerasInput)
    initialInceptionModule = InitialInceptionModule(kerasInput)
    intermeidateInceptionModule1 = IntermeidateInceptionModule(initialInceptionModule)
    intermeidateInceptionModule2 = IntermeidateInceptionModule(intermeidateInceptionModule1)
    intermeidateInceptionModule3 = IntermeidateInceptionModule(intermeidateInceptionModule2)
    firstBlock = Add()([residualModel1,intermeidateInceptionModule3])
    
    residualModel2=ResidualModel(firstBlock)
    intermeidateInceptionModule4 = IntermeidateInceptionModule(firstBlock)
    intermeidateInceptionModule5 = IntermeidateInceptionModule(intermeidateInceptionModule4)
    intermeidateInceptionModule6 = IntermeidateInceptionModule(intermeidateInceptionModule5)
    secondBlock = Add()([residualModel2,intermeidateInceptionModule3])
    
    averagePooling1D = AveragePooling1D(pool_size=1)(secondBlock)
    flatten = Flatten()(averagePooling1D)
    dense = Dense(4,activation='softmax')(flatten)
    model = Model(inputs=kerasInput, outputs=dense)
    model.compile(sgd, loss='categorical_crossentropy', metrics = ['accuracy'])
    return model
    
model = EEGInception()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1000, 22)]           0         []                            
                                                                                                  
 conv1d_2 (Conv1D)           (None, 1000, 48)             1104      ['input_1[0][0]']             
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 1000, 22)             0         ['input_1[0][0]']             
 D)                                                                                               
                                                                                                  
 conv1d_3 (Conv1D)           (None, 1000, 48)             57648     ['conv1d_2[0][0]']        

In [146]:
training_generator = Datasets2aSequence(x_train, y_train, 32)
validation_generator = Datasets2aSequence(x_test, y_test, 32)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001)
model.fit(training_generator, epochs=10, validation_data=validation_generator) 

Epoch 1/10
130/130 [==============================] - 52s 354ms/step - loss: 85.7676 - accuracy: 0.2850 - val_loss: 93.6472 - val_accuracy: 0.3025
Epoch 2/10
130/130 [==============================] - 46s 352ms/step - loss: 17.1684 - accuracy: 0.3890 - val_loss: 15.0290 - val_accuracy: 0.3757
Epoch 3/10
130/130 [==============================] - 46s 352ms/step - loss: 5.8583 - accuracy: 0.5561 - val_loss: 3.8161 - val_accuracy: 0.4509
Epoch 4/10
130/130 [==============================] - 46s 354ms/step - loss: 1.9382 - accuracy: 0.7321 - val_loss: 2.3401 - val_accuracy: 0.5723
Epoch 5/10
130/130 [==============================] - 46s 352ms/step - loss: 0.3192 - accuracy: 0.9151 - val_loss: 2.4202 - val_accuracy: 0.5761
Epoch 6/10
130/130 [==============================] - 46s 353ms/step - loss: 0.0735 - accuracy: 0.9740 - val_loss: 1.8361 - val_accuracy: 0.6262
Epoch 7/10
130/130 [==============================] - 46s 353ms/step - loss: 0.0087 - accuracy: 0.9986 - val_loss: 1.8060 - va

In [147]:
model.evaluate(Datasets2aSequence(x_evaluate, y_evaluate, 64))

9/9 [==============================] - 2s 153ms/step - loss: 1.6572 - accuracy: 0.6757


[1.657224178314209, 0.6756756901741028]

In [148]:
model.save('202309061325')

INFO:tensorflow:Assets written to: 202309061325/assets


INFO:tensorflow:Assets written to: 202309061325/assets
